Via a question to `neuprint@janelia.hhmi.org`, Josh writes:

>I'd like to know the number of t-bars on an SAG neuron that have pC1 neurites as a postsynaptic partner. Is there a way to query this?

Yes, this can be obtained via the [Python API][1].  (There might also be a way to obtain the numbers directly via a custom Cypher query, but it's probably easiest to analyze the results in Python.)
    
[1]: https://connectome-neuprint.github.io/neuprint-python/docs/

In [1]:
from neuprint import Client, fetch_neurons, fetch_synapse_connections, NeuronCriteria as NC
c = Client('neuprint.janelia.org', 'hemibrain:v1.2.1')

---

You can fetch the complete set of all synapse-synapse connections between SAG and pC1XX neurons with the following line:

In [3]:
conn_df = fetch_synapse_connections(NC(type='SAG'), NC(type='pC1.*', regex=True))
conn_df

,bodyId_pre,bodyId_post,roi_pre,roi_post,x_pre,y_pre,z_pre,x_post,y_post,z_post,confidence_pre,confidence_post
0,517587356,5813063587,SMP(R),SMP(R),21093,30794,8363,21090,30803,8379,0.936,0.998088
1,517587356,5813063587,SMP(R),SMP(R),21131,30815,8378,21098,30808,8384,0.996,0.651865
2,517587356,5813063587,SMP(R),SMP(R),17503,32189,9595,17525,32176,9609,0.993,0.682880
3,517587356,5813063587,SMP(R),SMP(R),20944,31314,8605,20932,31340,8597,0.995,0.742015
4,517587356,514850616,SMP(R),SMP(R),23010,26937,7682,23001,26920,7691,0.992,0.624554
...,...,...,...,...,...,...,...,...,...,...,...,...
947,5812981862,267214250,SMP(L),SMP(L),27860,25304,6243,27864,25310,6260,0.896,0.917000
948,5812981862,267214250,SMP(R),SMP(R),23073,29254,8971,23075,29242,8999,0.986,0.886246
949,5812981862,267214250,SMP(L),SMP(L),27843,25301,6235,27824,25326,6221,0.996,0.868097
950,5812981862,267214250,SMP(R),SMP(R),23513,27996,9049,23526,28011,9055,0.986,0.942423


---

BTW, If you'd like to see the types/instances of each `pre` and `post` neuron in that table, fetch their type information and merge it onto the table:

In [4]:
from neuprint import merge_neuron_properties
neuron_df, roi_counts = fetch_neurons(NC(type=['SAG', 'pC1.*'], regex=True))
conn_df = merge_neuron_properties(neuron_df, conn_df, ['type', 'instance'])
conn_df

,bodyId_pre,bodyId_post,roi_pre,roi_post,x_pre,y_pre,z_pre,x_post,y_post,z_post,confidence_pre,confidence_post,type_pre,instance_pre,type_post,instance_post
0,517587356,5813063587,SMP(R),SMP(R),21093,30794,8363,21090,30803,8379,0.936,0.998088,SAG,SAG,pC1d,pC1d_R
1,517587356,5813063587,SMP(R),SMP(R),21131,30815,8378,21098,30808,8384,0.996,0.651865,SAG,SAG,pC1d,pC1d_R
2,517587356,5813063587,SMP(R),SMP(R),17503,32189,9595,17525,32176,9609,0.993,0.682880,SAG,SAG,pC1d,pC1d_R
3,517587356,5813063587,SMP(R),SMP(R),20944,31314,8605,20932,31340,8597,0.995,0.742015,SAG,SAG,pC1d,pC1d_R
4,517587356,514850616,SMP(R),SMP(R),23010,26937,7682,23001,26920,7691,0.992,0.624554,SAG,SAG,pC1e,pC1e_R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
947,5812981862,267214250,SMP(L),SMP(L),27860,25304,6243,27864,25310,6260,0.896,0.917000,SAG,SAG,pC1b,pC1b_R
948,5812981862,267214250,SMP(R),SMP(R),23073,29254,8971,23075,29242,8999,0.986,0.886246,SAG,SAG,pC1b,pC1b_R
949,5812981862,267214250,SMP(L),SMP(L),27843,25301,6235,27824,25326,6221,0.996,0.868097,SAG,SAG,pC1b,pC1b_R
950,5812981862,267214250,SMP(R),SMP(R),23513,27996,9049,23526,28011,9055,0.986,0.942423,SAG,SAG,pC1b,pC1b_R


---

Anyway, from the connection table, you can probably get the numbers you’re after.  For instance, to obtain the number of unique tbars in that set:

In [27]:
conn_df[['x_pre', 'y_pre', 'z_pre']].drop_duplicates()

,x_pre,y_pre,z_pre
0,21093,30794,8363
1,21131,30815,8378
2,17503,32189,9595
3,20944,31314,8605
4,23010,26937,7682
...,...,...,...
936,25996,28512,12968
939,28449,31570,9875
942,27873,24630,6245
943,20131,22917,4710


---

Or to obtain the number of `pC1` post-synaptic partners each tbar has, use `groupby(...).size()`

In [6]:
conn_df.groupby(['bodyId_pre', 'x_pre', 'y_pre', 'z_pre']).size().sort_values(ascending=False)

bodyId_pre  x_pre  y_pre  z_pre
5812981862  24482  29158  11124    4
517587356   18925  30884  7705     4
            18996  31061  7730     4
5812981862  23571  28224  9657     4
517587356   23914  28660  10099    3
                                  ..
            28415  29565  8222     1
            28380  24954  5827     1
            28333  31492  9955     1
                   31445  9932     1
5812981862  32471  31409  8673     1
Length: 677, dtype: int64

---

...but those numbers don't necessarily prove that any tbars are partnered with the same downstream neuron multiple times.  They just indicate that some tbars have multiple `pC1` partner neurites, possibly from separate neurons.

---

Add `bodyId_post` to the grouping columns to see if there are any duplicated connections at a single tbar site:

In [7]:
conn_df.groupby(['bodyId_pre', 'bodyId_post', 'x_pre', 'y_pre', 'z_pre']).size().sort_values(ascending=False)

bodyId_pre  bodyId_post  x_pre  y_pre  z_pre
517587356   267214250    17719  30671  8132     1
5812981862  359744514    20409  31463  8346     1
                         17174  32166  9802     1
                         17418  32242  9697     1
                         17602  32236  9663     1
                                               ..
517587356   5813046951   20167  23619  4486     1
                         20306  23547  4515     1
                         20372  23528  4593     1
                         20415  21970  8714     1
5812981862  5813063587   19129  31218  7944     1
Length: 952, dtype: int64

---

So, it appears there are no `SAG` neurons which parter to the same `pC1` neuron multiple times from the same tbar site.

## Alternative: Cypher only

This Cypher query isn't well optimized, but it also yields the 677 tbars we found above.

In [28]:
q = """\
    MATCH (n:Neuron)-[:Contains]->(:SynapseSet)-[:Contains]->(tbar:Synapse)-[:SynapsesTo]->(psd:Synapse)<-[:Contains]-(:SynapseSet)-[:Contains]-(m:Neuron)

    WHERE
        n.type = 'SAG'
        and m.type =~ 'pC1.*'
        and tbar.type = 'pre'
        and psd.type = 'post'

    WITH DISTINCT tbar
    RETURN
      tbar.location.x as x,
      tbar.location.y as y,
      tbar.location.z as z
"""

c.fetch_custom(q)

,x,y,z
0,24906,29762,14056
1,24843,29646,13863
2,25953,28649,11829
3,24692,28624,12036
4,24762,28530,12017
...,...,...,...
672,21481,23591,5214
673,20429,23681,4740
674,20183,23384,4635
675,20868,21201,6603
